In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from preprocessor import Preprocessor
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf


In [31]:
train_preprocessor = Preprocessor()

data = pd.read_csv("../../data/wikipedia.csv")
# clean out where text is NaN
data = data[data.text.notna()]

In [32]:
doc_vectors = np.array([train_preprocessor.vectorize(text) for text in data.text])

In [33]:
features = tf.constant(doc_vectors, shape=(1,358,300))

labels = tf.constant(data.label.to_numpy(), shape=(1,358,1))

#train_dataset = tf.data.Dataset.from_tensor_slices((features,labels))
features_dataset = tf.data.Dataset.from_tensor_slices(features)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels)

train_dataset = tf.data.Dataset.zip((features_dataset, labels_dataset))

In [34]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(300, activation=tf.nn.relu, input_shape=(300,)),
  tf.keras.layers.Dense(1000, activation=tf.nn.relu),
  tf.keras.layers.Dense(1000, activation=tf.nn.relu),
  tf.keras.layers.Dense(100)
])

In [35]:
loss_object = tf.keras.losses.MeanSquaredError()

def get_random_result(label, labels, predictions, predictions_cache):
  label = label[0]
  # TODO: Check if the index of the x value we are testing is not exactly the same as the one we are returning as y
  if(label not in predictions_cache):
    predictions_cache[label] = []
    for idx2, value in enumerate(labels.numpy()):
        if(value == label):
            predictions_cache[label].append(predictions[idx2])
  return predictions_cache[label][random.randint(0,len(predictions_cache[label])-1)]

def get_new_y(predictions, labels):
  predictions_cache = {}
  new_y = []
  for idx, val in enumerate(predictions.numpy()):
      label = labels[idx].numpy()
      new_y.append(get_random_result(label, labels, predictions, predictions_cache))
  return new_y

def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)
  new_y = get_new_y(y_, y)

  return loss_object(y_true=new_y, y_pred=y_)

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [36]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [37]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
num_epochs = 5000
import time
for epoch in range(num_epochs):
  start = time.time()
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.MeanAbsoluteError()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(x, training=True)
    new_y = get_new_y(predictions, y)
    epoch_accuracy.update_state(new_y, predictions)

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  end = time.time()
  print("Epoch {:03d}: Loss: {:.3f}, MAE: {:.3}, time: {:.8f}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result(), end - start))

400768
Epoch 4636: Loss: 0.003, MAE: 0.0466, time: 0.15616727
Epoch 4637: Loss: 0.004, MAE: 0.0449, time: 0.15657830
Epoch 4638: Loss: 0.004, MAE: 0.0459, time: 0.15779877
Epoch 4639: Loss: 0.004, MAE: 0.0455, time: 0.16673064
Epoch 4640: Loss: 0.004, MAE: 0.0457, time: 0.16292477
Epoch 4641: Loss: 0.003, MAE: 0.0458, time: 0.17294025
Epoch 4642: Loss: 0.004, MAE: 0.0452, time: 0.14724588
Epoch 4643: Loss: 0.003, MAE: 0.0452, time: 0.14706230
Epoch 4644: Loss: 0.003, MAE: 0.0452, time: 0.15099406
Epoch 4645: Loss: 0.004, MAE: 0.046, time: 0.15485501
Epoch 4646: Loss: 0.003, MAE: 0.0453, time: 0.15872765
Epoch 4647: Loss: 0.003, MAE: 0.0457, time: 0.14951921
Epoch 4648: Loss: 0.004, MAE: 0.0456, time: 0.15277338
Epoch 4649: Loss: 0.004, MAE: 0.0466, time: 0.16581988
Epoch 4650: Loss: 0.004, MAE: 0.0457, time: 0.15060663
Epoch 4651: Loss: 0.004, MAE: 0.0457, time: 0.15703106
Epoch 4652: Loss: 0.004, MAE: 0.0457, time: 0.15396547
Epoch 4653: Loss: 0.003, MAE: 0.0458, time: 0.15166473
Epoc

In [38]:
a = train_preprocessor.vectorize("Teiglinge nach der Stückgare")
b = train_preprocessor.vectorize("Das Gerät wird gleichzeitig gezogen oder vom Traktor getragen und über die Zapfwelle oder hydraulisch angetrieben")
c = train_preprocessor.vectorize("Portionsstücke werden abgeteilt; der Teig wird „ausgebrochen“")
d = train_preprocessor.vectorize("Für aktuelle Landtechnik sind die führenden Zeitschriften:")

a = train_preprocessor.vectorize("Brot ist ein leckeres im Ofen gebackenes Rezept")
b = train_preprocessor.vectorize("In der Landwirdschaft gibt es viele Traktoren")
c = train_preprocessor.vectorize("Lecker sind z.B. Roggenbrot oder Walnussbrötchen")
d = train_preprocessor.vectorize("Viele Bauern leben neben ihren Garagen um leicht ihre Zylinder reinigen zu können")

a = train_preprocessor.vectorize("Der Brauch, Brot und Salz etwa zum Bezug einer neuen Wohnung oder zur Hochzeit zu schenken, soll Wohlstand sichern.")
b = train_preprocessor.vectorize("Die zunehmende Sammelleidenschaft für historische Landmaschinen in der Bevölkerung zog auch die Ausgabe diverser Fachzeitschriften nach sich, die nach langer Dominanz klassischer Vereinsnachrichten den Markt um professionelle journalistische Produkte bereicherten. Diese Publikationen sind meist im Bahnhofsbuchhandel erhältlich. Sie informieren im Zeitschriftenformat und in Farbe über Restaurierungsprojekte, Vereinsaktivitäten, Schleppertreffen und vieles mehr.")
c = train_preprocessor.vectorize("„Brot wird ganz oder teilweise aus Getreide und/oder Getreideerzeugnissen, meist nach Zugabe von Flüssigkeit, sowie von anderen Lebensmitteln (z. B. Leguminosen-, Kartoffelerzeugnisse) in der Regel durch Kneten, Formen, Lockern, Backen oder Heißextrudieren des Brotteiges hergestellt. Brot enthält weniger als 10 Gewichtsteile Fett und/oder Zuckerarten auf 90 Gewichtsteile Getreide und/oder Getreideerzeugnisse.")
d = train_preprocessor.vectorize("Um Unfälle mit landwirtschaftlichen Zugmaschinen abzuschwächen oder ganz zu vermeiden, empfiehlt die UDV eine bessere Schulung junger Traktorfahrer. Ferner sollten alle Verkehrsteilnehmer besser über das besondere Unfallrisiko mit Schleppern aufgeklärt werden. Das Signalbild von Traktoren mit und ohne Anhänger müsse verbessert werden, beispielsweise durch die Zulassung von Rundum-Leuchten, Reflexfolien, Begrenzungsleuchten und Konturmarkierungen. Heckleuchten und Blinker sollten sowohl größer als auch stabiler sein.")

trained = model(tf.constant([a,b,c,d]), training=False)

diff1 = np.linalg.norm(trained[0]-trained[2])
diff2 = np.linalg.norm(trained[1]-trained[3])
diff3 = np.linalg.norm(trained[0]-trained[3])
diff4 = np.linalg.norm(trained[1]-trained[2])

print("Should be low")
print(diff1)
print(diff2)

print("Should be higher")
print(diff3)
print(diff4)

Should be low
0.7402869
0.4161326
Should be higher
0.6740105
0.5865265
